<a href="https://colab.research.google.com/github/cheungkelly/DATA_301/blob/main/Day_14_Training_and_Test_Errors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**training data -> training set & validation,
predicted y^ actual y => MSE**

# Training and Test Errors

In [ ]:
import pandas as pd
import numpy as np

**Try to use scikit-learn whenever possible.**

## Ames Housing Data

In [ ]:
df_ames = pd.read_csv("http://dlsun.github.io/pods/data/AmesHousing.txt", sep = "\t")

df_ames

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2925,2926,923275080,80,RL,37.0,7937,Pave,NaN,IR1,Lvl,...,0,NaN,GdPrv,NaN,0,3,2006,WD,Normal,142500
2926,2927,923276100,20,RL,NaN,8885,Pave,NaN,IR1,Low,...,0,NaN,MnPrv,NaN,0,6,2006,WD,Normal,131000
2927,2928,923400125,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal,132000
2928,2929,924100070,20,RL,77.0,10010,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,4,2006,WD,Normal,170000


1\. Fit a $10$-nearest neighbors model to predict **SalePrice** using **Bldg Type** as the only feature.

In [ ]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

ct = make_column_transformer(
    (OneHotEncoder(), ["Bldg Type"]),
    remainder="drop"  # all other columns in X will be dropped.
)
ct
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import KNeighborsRegressor

pipeline = make_pipeline(
    ct,
    KNeighborsRegressor(n_neighbors=10)
)

pipeline.fit(X=df_ames[["Bldg Type"]],
             y=df_ames["SalePrice"])

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['Bldg Type'])])),
                ('kneighborsregressor', KNeighborsRegressor(n_neighbors=10))])

2\. Calculate the **training error** of this model. Try a few different performance metrics.

In [ ]:
y_train_ = pipeline.predict(X=df_ames[["Bldg Type"]])
y_train_

array([185170., 185170., 185170., ..., 185170., 185170., 185170.])

In [ ]:
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(df_ames["SalePrice"], y_train_)
rmse = np.sqrt(mse)
rmse

78700.78641236422

3\. Repeat the above process to calculate the training error for $k=1, 2, \ldots, 10$. Which value of $k$ gives the smallest training error? Does that necessarily mean this is the best value of $k$? Discuss with your partner.

In [ ]:
# YOUR CODE HERE. ADD CELLS AS NEEDED
errors = []

for k in [1,2,3,4,5,6,7,8,9,10]:
    pipeline = make_pipeline(
        ct,
        KNeighborsRegressor(n_neighbors=k)
    )

    # Convert the reshaped array back to DataFrame
    X_train = pd.DataFrame(df_ames["Bldg Type"].values, columns=["Bldg Type"])
    y_train = df_ames["SalePrice"]

    pipeline.fit(X=X_train, y=y_train)

    y_pred = pipeline.predict(X_train)
    mse = mean_squared_error(y_train, y_pred)
    rmse = np.sqrt(mse)

    print(f'training error for k = {k} is {rmse}')
    errors.append(rmse)

# Get the k value with the smallest error
min_error_k = errors.index(min(errors)) + 1
print(f"The smallest training error was with k = {min_error_k}")

training error for k = 1 is 83869.12582123668
training error for k = 2 is 82461.56991333194
training error for k = 3 is 81459.84807307809
training error for k = 4 is 79077.36953503109
training error for k = 5 is 78841.98384073898
training error for k = 6 is 78718.47565629905
training error for k = 7 is 78666.84969111855
training error for k = 8 is 78813.2596795962
training error for k = 9 is 78781.76304262652
training error for k = 10 is 78700.78641236422
The smallest training error was with k = 7


4\. Return to the model in part 1. Now estimate the **test error** **of** the model using cross-validation. Try a few different performance metrics.

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(pipeline,
                         X=df_ames[["Bldg Type"]],
                         y=df_ames["SalePrice"],
                         scoring="neg_mean_squared_error",
                         cv=2)
scores

array([-6.18017740e+09, -6.42290232e+09])

In [ ]:
np.sqrt(-scores)

array([78614.10436122, 80143.01165027])

In [ ]:
np.sqrt(-scores).mean()

79378.55800574775

5\. Now, define a 10-nearest neighbors model to predict **SalePrice** using **Neighborhood** as the only feature. Try to estimate the test error of this model using cross validation.

You will get an error. Can you figure out why this error occurs? Can you figure out how to fix it?

In [ ]:
# YOUR CODE HERE. ADD CELLS AS NEEDED
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

ct = make_column_transformer(
    (OneHotEncoder(), ["Neighborhood"]),
    remainder="drop"  # all other columns in X will be dropped.
)
ct
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import KNeighborsRegressor

pipeline = make_pipeline(
    ct,
    KNeighborsRegressor(n_neighbors=10)
)

pipeline.fit(X=df_ames[["Neighborhood"]],
             y=df_ames["SalePrice"])

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['Neighborhood'])])),
                ('kneighborsregressor', KNeighborsRegressor(n_neighbors=10))])

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(pipeline,
                         X=df_ames[["Neighborhood"]],
                         y=df_ames["SalePrice"],
                         scoring="neg_mean_squared_error",
                         cv=2)
scores

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:794: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 115, in __call__
    score = scorer._score(cached_call, estimator, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 276, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 73, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 480, in predict
    Xt = transform.transform(Xt)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils/_set_output.py", line 140, in wrapped
    data_to_wrap = f(self, X, *args, **kwargs)
  File "/usr/local/

array([-3.36865314e+09,             nan])

6\. Recall that in a previous notebook we fit a 10-nearest neighbors regression model that predicts the price (just **SalePrice**, not log) of a home using square footage (**Gr Liv Area**), number of bedrooms (**Bedroom AbvGr**), number of full bathrooms (**Full Bath**), number of half bathrooms (**Half Bath**), and **Neighborhood**. Fit this model and estimate its test error using cross-validation. Try a few different performance metrics.

In [ ]:
from sklearn.preprocessing import StandardScaler

# Features to use
features = ["Gr Liv Area", "Bedroom AbvGr", "Full Bath", "Half Bath", "Neighborhood"]

# Column transformer setup
ct_full = make_column_transformer(
    (StandardScaler(), ["Gr Liv Area", "Bedroom AbvGr", "Full Bath", "Half Bath"]),
    (OneHotEncoder(handle_unknown='ignore'), ["Neighborhood"]),
    remainder="drop"
)

# Create a pipeline
pipeline_full = make_pipeline(
    ct_full,
    KNeighborsRegressor(n_neighbors=10)
)

# Mean squared error
mse_scores = cross_val_score(pipeline_full, df_ames[features], df_ames["SalePrice"], cv=5, scoring='neg_mean_squared_error')
mse_avg = -mse_scores.mean()

# Mean absolute error
mae_scores = cross_val_score(pipeline_full, df_ames[features], df_ames["SalePrice"], cv=5, scoring='neg_mean_absolute_error')
mae_avg = -mae_scores.mean()

# R^2 score
r2_scores = cross_val_score(pipeline_full, df_ames[features], df_ames["SalePrice"], cv=5, scoring='r2')
r2_avg = r2_scores.mean()

print(f"Average MSE: {mse_avg}")
print(f"Average MAE: {mae_avg}")
print(f"Average R^2 Score: {r2_avg}")

Average MSE: 1600557376.3328702
Average MAE: 25598.395597269624
Average R^2 Score: 0.7470728767749495


7\. Repeat the process in part 6 to fit $k$-nearest neighbors regression models for several values of $k$ (say $k=1, \ldots, 20$). Which value of $k$ produces the best test error? Try a few different performance metrics; does the best value of $k$ depend on the metric?

In [ ]:
# prompt:


# 7\. Repeat the process in part 6 to fit $k$-nearest neighbors regression models for several values of $k$ (say $k=1, \ldots, 20$). Which value of $k$ produces the best test error? Try a few different performance metrics; does the best value of $k$ depend on the metric?
# YOUR CODE HERE. ADD CELLS AS NEEDED

# Create a list of k values to try
k_values = list(range(1, 21))

# Create a list to store the test errors for each k value
test_errors = []

# Fit a k-nearest neighbors model for each k value and calculate the test error
for k in k_values:
    pipeline_full = make_pipeline(
        ct_full,
        KNeighborsRegressor(n_neighbors=k)
    )

    test_error = cross_val_score(pipeline_full, df_ames[features], df_ames["SalePrice"], cv=5, scoring='neg_mean_squared_error')
    test_errors.append(-test_error.mean())

# Find the k value with the smallest test error
best_k = k_values[np.argmin(test_errors)]

# Print the best k value and the corresponding test error
print(f"The best k value is {best_k} with a test error of {test_errors[best_k - 1]}")


The best k value is 6 with a test error of 1546411924.9619267
